# Notebook: Generation of image and label datasets

Generate datasets of roof segment labels for aerial imagery derived from CityGML semantic 3D city models for semantic segmentation.

In [ ]:
import os
from shutil import copy2
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.mask
import rasterio.features
import shapely
import cv2
import time

from owslib.wms import WebMapService
from owslib.util import Authentication

# Importing WMS username and password
from wms_config import username as wms_username, password as wms_password

from dataset_creation_utils import *

# Data import and preprocessing

## Special case: How to create samples at wbg_m (aka _small-manu_) centroids with 3DCityDB segments

For the configuration _small-auto_, it is necessary to create samples around the _small-manu_ building centroids using the _small-auto_ segment geometries.

1. Load wbg_m / small-manu segments from "Import segments" section.
2. Convert to EPSG:25832
3. Derive buildings, centroids, and buffers.
4. Filter buildings: Run the cell at section "Perform data split > Prepare hand-labeled dataset: Filter out buildings without image/label pair"
5. Use "Import segments" section again, this time to load 3DCityDB segments of Wartenberg, small-auto.
6. DON'T recompute the buildings, keep the pvbackend building dataframe.
7. Run sections to create dataset using the Web Map Service.

## Import segments and set up GeoDataFrame

In [ ]:
# select configuration
scenario = "small-manu"

# set input data parameters:
# tables with roof segment geometries
segments_dirpath = "segments"
segments_filenames = {
    "small-manu":                         "segments_small-manu.csv",
    "small-auto":                         "segments_small-auto.csv",
    "large-auto":                         "segments_large-auto.csv",  # Bavaria dataset including column indicating roof generation method
    "large-auto_legacy":                  "segments_large-auto_legacy.csv",
    "large-auto_legacy_Erding_Freising":  "segments_large-auto_legacy_Erding_Freising.csv",
    "large-auto_legacy_Munich":           "segments_large-auto_legacy_Munich.csv",
    "large-auto_legacy_Rural":            "segments_large-auto_legacy_Rural.csv"
}

# CRS of roof segment data
# epsg:25832 for LDBV 3DCityDB data and aerial imagery
# epsg:4326 for Google Satellite data and imagery
segments_crss = {
    "small-manu":                         "epsg:4326",
    "small-auto":                         "epsg:25832",
    "large-auto":                         "epsg:25832",
    "large-auto_legacy":                  "epsg:25832",
    "large-auto_legacy_Erding_Freising":  "epsg:25832",
    "large-auto_legacy_Munich":           "epsg:25832",
    "large-auto_legacy_Rural":            "epsg:25832",
}

# set segments filepath and CRS
segments_filepath = os.path.join(segments_dirpath, segments_filenames[scenario])
segments_crs = segments_crss[scenario]

In [ ]:
# read data
segments = gpd.read_file(segments_filepath, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")
segments.crs = segments_crs

# some data corrections, apply numeric data types
segments.loc[segments["azimuth"] == "NaN", "azimuth"] = ""
segments.loc[segments["slope"] == "NaN", "slope"] = ""
segments["azimuth"] = pd.to_numeric(segments["azimuth"])
segments["slope"] = pd.to_numeric(segments["slope"])
segments["b_id"] = segments["b_id"].astype(int)
segments["sg_id"] = segments["sg_id"].astype(int)

# derive orientation classes from azimuth angle: parameter setup
# directions / angles used here: north = -180°, east = -90°, south = 0°, west = 90°
bin_width = 22.5
angle_bins = [-180 - bin_width/2 + k * bin_width for k in range(0,18)]
labels = ["n", "nne", "ne", "ene", "e", "ese", "se", "sse", "s", "ssw", "sw", "wsw", "w", "wnw", "nw", "nnw", "n"]

# sort segment azimuth angles into 16 bins
segments["orientation"] = pd.Series(pd.cut(segments["azimuth"], angle_bins, labels = labels, ordered=False))

# add category "flat" to orientation categorical
segments["orientation"] = segments["orientation"].cat.add_categories("flat")

# turn NaN orientation resulting from NaN azimuth into "flat"
segments.loc[pd.isna(segments["orientation"]), "orientation"] = "flat"

# dict to translate into numerical orientation classes
# note: background has class value 0, this value is assigned automatically to image areas
# without segment by rasterio.features.rasterize() used in save_png_labels().
orientation_dict = {
    "n": 1,
    "nne": 2,
    "ne": 3,
    "ene": 4,
    "e": 5,
    "ese": 6,
    "se": 7,
    "sse": 8,
    "s": 9,
    "ssw": 10,
    "sw": 11,
    "wsw": 12,
    "w": 13,
    "wnw": 14,
    "nw": 15,
    "nnw": 16,
    "flat": 17
}

# Alternative class values with background class getting value 17 instead of 0.
# If you use them to generate a new dataset, remember to change fill value for background class to 17 (in function save_png_labels).
"""
orientation_dict = {
    "n": 0,
    "nne": 1,
    "ne": 2,
    "ene": 3,
    "e": 4,
    "ese": 5,
    "se": 6,
    "sse": 7,
    "s": 8,
    "ssw": 9,
    "sw": 10,
    "wsw": 11,
    "w": 12,
    "wnw": 13,
    "nw": 14,
    "nnw": 15,
    "flat": 16
}
"""

# assign numerical orientation classes
segments['orientation_num'] = segments['orientation'].map(orientation_dict)

In [ ]:
# only apply for pvbackend-segments (orignal hand-labelled segments) for execution of data split
# to make sure the buffers of the buildings correspond to the actual image size:
# projection to epsg:25832
if segments.crs == "epsg:4326": segments = segments.to_crs("epsg:25832")

## Derive buildings from segments

Two options (A) and (B) depending on whether the column "method", indicative of the roof generation method / algorithm, is available or not.

### A) If column "method" is not available:

In [ ]:
# dissolve / aggregate segment geometries by building id
buildings = segments[["b_id", "geometry"]].dissolve(by = "b_id", as_index = False)

### B) If column "method" is available:

Column method: roof geometry generation method, generic attribute from 3D city data.

In [ ]:
# dissolve / aggregate segment geometries by building id
buildings = segments[["b_id", "method", "geometry"]].dissolve(by = "b_id", as_index = False)

In [ ]:
segments["method"].value_counts(), segments["method"].count()

In [ ]:
buildings["method"].value_counts(), buildings["method"].count(), buildings.loc[buildings["method"].isin(["3210", "3220", "3100"]), "method"].count()

## Compute centroids and sample-sized buffers

In [ ]:
# set params:
# - img_size_px: square image side length in pixels
# - img_res_m_per_pix: resolution of imagery in meters per pixel
img_size_px = 256
img_res_m_per_px = 0.2
img_size_m = img_size_px * img_res_m_per_px

In [ ]:
# centroids
buildings["centroid"] = buildings.centroid

# create square buffers around buildings to crop / retrieve aerial image
buildings["buffer"] = buildings["centroid"].buffer(img_size_m/2, cap_style=3)

# number of buildings that are not contained within the image-sized buffer
sum(buildings["buffer"].contains(buildings["geometry"]) == False)

## Remove samples with false labels (default flat roofs)

Only if column "method" is available. Two options (A) or (B). Variant (B) was used to generate the dataset bv_nfl aka _large-auto_. Variant (A), which is more exact, was implemented afterwards but the improvement was deemed too minor (gain of 260 samples) to re-do the data split and re-train the model.

### A) Exact method

First uses sindex.query to check for intersecting envelopes and in a second step uses sjoin to check for exact intersections. Takes longer, but saves some buildings from unnecessarily being sorted out.

In [ ]:
samples_containing_false_labels = np.array([], dtype = np.int64)

# list of roof generation methods that indicate assignemnt of a default flat roof
undesirable_geom_methods = ["3210", "3220", "3100"]

for index, building in buildings.iterrows():
    
    # identify all buildings that intersect the current building's buffer
    intersecting_buildings = buildings.iloc[buildings.sindex.query(building["buffer"])]
    
    # if among these there are any with undesired roof geometry generation method
    if len(intersecting_buildings[intersecting_buildings["method"].isin(undesirable_geom_methods)]) > 0:
        
        buffer_gdf = gpd.GeoDataFrame({"geometry": building["buffer"]}, index = [0])
        buffer_gdf.crs = segments_crs
        
        intersecting_buildings_2 = intersecting_buildings.sjoin(buffer_gdf, how = "inner", predicate = "intersects")
        
        if len(intersecting_buildings_2[intersecting_buildings_2["method"].isin(undesirable_geom_methods)]) > 0:
        
            samples_containing_false_labels = np.append(
                samples_containing_false_labels,
                building["b_id"]
            )

In [ ]:
samples_containing_false_labels.shape

### B) Not quite as exact method

Only uses sindex.query to check for intersecting envelopes. Faster, but leads to sorting out of a few samples that could be kept.

In [ ]:
samples_containing_false_labels = np.array([], dtype = np.int64)
undesirable_geom_methods = ["3210", "3220", "3100"]

for index, building in buildings.iterrows():
    
    # identify all buildings that intersect the current building's buffer
    intersecting_buildings = buildings.iloc[buildings.sindex.query(building["buffer"])]
    
    # if among these there are any with undesired roof geometry generation method
    if len(intersecting_buildings[intersecting_buildings["method"].isin(undesirable_geom_methods)]) > 0:
        
        samples_containing_false_labels = np.append(
            samples_containing_false_labels,
            building["b_id"]
        )

In [ ]:
samples_containing_false_labels.shape

Note:

If only using the sindex.query as intersection method, which only intersects envelopes / bounding boxes of the features, the resulting number of buildings within whose sample extent there are false labels is 28560.

If additionally using sjoin as intersection method after sindex.query found intersecting envelopes of false labels to sort out any buildings with false labels whose envelope intersects the buffer but whose exact geometry does not, the resulting number of buildings within whose sample extent there are false labels is 28300.

The difference is, therefore, quite negligible.

Next cell: Filters out undesired samples, must be executed after identifying them using (A) or (B).

In [ ]:
buildings_complete = buildings
buildings = buildings[~buildings["b_id"].isin(samples_containing_false_labels)]

# Plot study area

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
plt.rc("font", size = 14)

In [ ]:
plot_dir = "Plots"
save_fig = False

plotsize_px = 3000
figsize_inches = 10
dpi = plotsize_px / figsize_inches

fig, ax = plt.subplots(
    figsize = (figsize_inches, figsize_inches),
    dpi = dpi, 
    constrained_layout = True
)

buildings.plot(ax = ax, color = "black")
if save_fig:
    fig.savefig(os.path.join(plot_dir, "".join(["buildings_", scenario, "_", str(plotsize_px), "_", str(figsize_inches), ".png"])))

# Create dataset of images and labels

In [ ]:
# make sure Windows' controlled folder access allows Python to write in folder if protected
dataset_dirpath = r"datasets"
dataset_dirname = "dataset_wbg_a"
images_dirpath = os.path.join(dataset_dirpath, dataset_dirname, "images")
labels_dirpath = os.path.join(dataset_dirpath, dataset_dirname, "labels")
if not os.path.exists(images_dirpath): os.makedirs(images_dirpath)
if not os.path.exists(labels_dirpath): os.makedirs(labels_dirpath)

## 3D city data based labels: Using DOP WMS

In [ ]:
authenticate = Authentication()

# connect to Web Map Service
wms = WebMapService('https://geoservices.bayern.de/wms/v2/ogc_dop20.cgi?',
                    version = '1.1.1',
                    xml = None, 
                    username = wms_username,
                    password = wms_password, 
                    parse_remote_metadata = False, 
                    timeout = 180,
                    headers = None, 
                    auth = authenticate)

In [ ]:
begin_total = time.time()

start = 0
stop = len(buildings)

for index, row in buildings[start:stop].iterrows():
    
    #begin = time.time()
    
    # obtain crop of aerial image at building location from WMS
    tile = wms.getmap(layers = ['by_dop20c'],
                      styles = ['default'],
                      srs = 'EPSG:25832',
                      bbox = row["buffer"].bounds,
                      size = (img_size_px, img_size_px),
                      format = 'image/tiff',
                      transparent = True)
    
    image_filename = str(row["b_id"]) + ".tif"
    image_filepath = os.path.join(images_dirpath, image_filename)
    
    out = open(image_filepath, "wb")
    out.write(tile.read())
    out.close()
    
    #print("Fetched and saved WMS image crop in " + str(time.time() - begin) + " seconds.")
    #begin = time.time()
    
    # get segments that intersect the building's buffer
    label_segments = get_label_segments_sindex(segments = segments,
                                               crop_geometry = row["buffer"])
    
    #print("Fetched required segments in " + str(time.time() - begin) + " seconds.")
    #begin = time.time()
    
    # Attempt to obtain transform without having to read the corresponding image file - not functional
    #bbox = row["buffer"].bounds
    #transform = rasterio.transform.from_bounds(bbox[1],bbox[0],bbox[3],bbox[2], img_size_px, img_size_px)
    
    # rasterize these segments and save them as label pngs
    with rasterio.open(image_filepath) as image:
        save_png_labels(segments_geometry = label_segments["geometry"],
                        segments_orientation = label_segments["orientation_num"],
                        shape = (img_size_px, img_size_px),
                        transform = image.transform,
                        dirpath = labels_dirpath,
                        filename = str(row["b_id"]))
    
    #print("Saved required segments in " + str(time.time() - begin) + " seconds.")
    
    print(f"Sample {str(index+1)} done.", end = "\r")
    
end_total = time.time()
seconds = end_total - begin_total

print("Created {} training samples in {} seconds.".format(str(stop-start), str(seconds)))

## 3D city data based labels: Using local DOP mosaic

In [ ]:
# set params
save_img_as_png = False
save_img_as_tif = True
in_filepath = r"mosaic.tif"

# convert buffer geometries to json (required by rasterio.mask)
buildings["buffer_json"] = buildings["buffer"].apply(lambda x: shapely.geometry.mapping(x))

# open aerial image raster mosaic
in_tif = rasterio.open(in_filepath)

for index, row in buildings.iterrows():
   
    # crop aerial image with building's buffer
    out_img, out_meta = get_image_crop(crop_geometry_json = row["buffer_json"],
                                       in_tif = in_tif)
    
    # save crop as png
    if save_img_as_png:
        save_png_image(img_rio = out_img,
                       dirpath = images_dirpath,
                       filename = str(row["b_id"]))
    
    # save crop as tif
    if save_img_as_tif:
        save_tif_image(img_rio = out_img,
                       img_meta = out_meta,
                       dirpath = images_dirpath,
                       filename = str(row["b_id"]))
    
    # get segments that intersect the building's buffer
    label_segments = get_label_segments(segments = segments,
                                        crop_geometry = row["buffer"],
                                        crop_crs = segments_crs)
    
    # rasterize these segments and save them as label pngs
    save_png_labels(segments_geometry = label_segments["geometry"],
                    segments_orientation = label_segments["orientation_num"],
                    shape = (out_meta["width"], out_meta["height"]),
                    transform = out_meta["transform"],
                    dirpath = labels_dirpath,
                    filename = str(row["b_id"]))
    
    print(f"Sample {str(index+1)} done.", end = "\r")
                                        
in_tif.close()

print("Done!")

# Create random subsets of datasets

Not used for paper, may be used for testing or other purposes.

## Take random sample

In [ ]:
buildings_subset = buildings.sample(n = 1878)

In [ ]:
m = buildings.explore()
buildings_subset.explore(m = m, color = "red")

In [ ]:
# remove columns "centroid" and "buffer" that inhibit functioning of to_file-function
buildings_subset = buildings_subset.drop(["centroid", "buffer"], axis = "columns")

In [ ]:
buildings_subset.to_file(os.path.join(segments_dirpath, "wbg_buildings_subset_20220117.geojson"), driver = "GeoJSON")

## Relocate subset images and labels

In [ ]:
dataset_dirpath = r"datasets"
dataset_dirname_src = "dataset_wbg_v4"
dataset_dirname_dst = "wbg_v4_subset"
images_dirpath_src = os.path.join(dataset_dirpath, dataset_dirname_src, "images")
labels_dirpath_src = os.path.join(dataset_dirpath, dataset_dirname_src, "labels")
images_dirpath_dst = os.path.join(dataset_dirpath, dataset_dirname_dst, "images")
labels_dirpath_dst = os.path.join(dataset_dirpath, dataset_dirname_dst, "labels")
images_filetype = ".tif"
labels_filetype = ".png"
if not os.path.exists(images_dirpath_dst): os.makedirs(images_dirpath_dst)
if not os.path.exists(labels_dirpath_dst): os.makedirs(labels_dirpath_dst)

In [ ]:
relocate_samples(
    ids = buildings_subset["b_id"],
    filetype = images_filetype,
    dirpath_src = images_dirpath_src,
    dirpath_dst = images_dirpath_dst
)

In [ ]:
relocate_samples(
    ids = buildings_subset["b_id"],
    filetype = labels_filetype,
    dirpath_src = labels_dirpath_src,
    dirpath_dst = labels_dirpath_dst
)

## Update buildings dataframe to perform data split

In [ ]:
# reconstruct buildings dataframe to perform data split
buildings = buildings_subset

In [ ]:
# centroids
buildings["centroid"] = buildings.centroid

# create square buffers around buildings to crop / retrieve aerial image
buildings["buffer"] = buildings["centroid"].buffer(img_size_m/2, cap_style=3)

# number of buildings that are not contained within the image-sized buffer
sum(buildings["buffer"].contains(buildings["geometry"]) == False)

# Perform data split

## Prepare hand-labeled dataset: Filter out buildings without image/label pair

Only execute for manually labeled dataset (wbg_m aka _small-manu_).

In [ ]:
# from all buildings in pvbackend-database, select only those for which
# an image-label-pair exists in the data provided by Sebastian.
images_dirpath_src = r"images_roof_centered_png"
images_filetype = ".png"
image_filenames = [file for file in os.listdir(images_dirpath_src) if file.lower().endswith(images_filetype)]

image_ids = [int(fn[:-4]) for fn in image_filenames]
b_ids = buildings[buildings["b_id"].isin(image_ids)]["b_id"].tolist()

buildings = buildings[buildings["b_id"].isin(b_ids)]

In [ ]:
# only to inform: number of segments that belong to these filtered buildings
segments[segments["b_id"].isin(buildings["b_id"])].shape

## Split building dataframe into subsets

In [ ]:
# Read the right table with locations and buildings numbers for validation
# and test sets, depending on scenario that is being processed.

val_test_points_dirpath = "val_test_locations"
val_test_points_filenames = {
    "small-manu":                         "val_test_locations_small-manu_small-auto.csv",
    "small-auto":                         "val_test_locations_small-manu_small-auto.csv"
    "large-auto":                         "val_test_locations_large-auto.csv",
    "small-auto_legacy":                  "val_test_locations_small-auto_legacy.csv",  # If small-auto dataset uses small-auto building centroids instead of small-manu building centroids
    "large-auto_legacy":                  "val_test_locations_large-auto_legacy.csv",
    "large-auto_legacy_Erding_Freising":  "val_test_locations_large-auto_legacy_Erding_Freising.csv",
    "large-auto_legacy_Munich":           "val_test_locations_large-auto_legacy_Munich.csv",
    "large-auto_legacy_Rural":            "val_test_locations_large-auto_legacy_Rural.csv",
}

val_test_points_filepath = os.path.join(val_test_points_dirpath, val_test_points_filenames[scenario])
val_test_points = gpd.read_file(val_test_points_filepath, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")
val_test_points.crs = "epsg:25832"
val_test_points["number"] = val_test_points["number"].astype(int)

In [ ]:
test_set, test_areas, _ = get_data_subset(points = val_test_points.geometry[val_test_points["type"] == "test"],
                                          numbers = val_test_points.loc[val_test_points["type"] == "test", "number"],
                                          buildings = buildings)

buildings_except_test_set_overlap = remove_overlapping_buildings(buildings, test_set)

val_set, val_areas, _ = get_data_subset(points = val_test_points.geometry[val_test_points["type"] == "val"],
                                        numbers = val_test_points.loc[val_test_points["type"] == "val", "number"],
                                        buildings = buildings_except_test_set_overlap)

train_set = remove_overlapping_buildings(buildings_except_test_set_overlap, val_set)

In [ ]:
# Store the found subsets as geojson files
train_set_geojson = train_set.drop(["centroid", "buffer"], axis = "columns")
val_set_geojson = val_set.drop(["centroid", "buffer"], axis = "columns")
test_set_geojson = test_set.drop(["centroid", "buffer"], axis = "columns")

train_set_geojson.to_file("".join([scenario, "_train_set.geojson"]), driver = "GeoJSON")
val_set_geojson.to_file("".join([scenario, "_val_set.geojson"]), driver = "GeoJSON")
test_set_geojson.to_file("".join([scenario, "_test_set.geojson"]), driver = "GeoJSON")

val_areas.to_file("".join([scenario, "_val_areas.geojson"]), driver = "GeoJSON")
test_areas.to_file("".join([scenario, "_test_areas.geojson"]), driver = "GeoJSON")

## Relocate images and labels accordingly

In [ ]:
dataset_dirpath = r"datasets"
dataset_dirname = "dataset_bv_complete"
images_dirpath = os.path.join(dataset_dirpath, dataset_dirname, "images")
labels_dirpath = os.path.join(dataset_dirpath, dataset_dirname, "labels")
images_filetype = ".tif"
labels_filetype = ".png"

if not os.path.exists(images_dirpath): raise RuntimeError("Could not find images directory.")
if not os.path.exists(labels_dirpath): raise RuntimeError("Could not find labels directory.")

ids = [train_set["b_id"], train_set["b_id"], val_set["b_id"], val_set["b_id"], test_set["b_id"], test_set["b_id"]]
filetypes = 3 * [images_filetype, labels_filetype]
dirpaths_src = 3 * [images_dirpath, labels_dirpath]
dirnames_dst = ["train", "trainannot", "val", "valannot", "test", "testannot"]
dirpaths_dst = [os.path.join(dataset_dirpath, dataset_dirname, dirname) for dirname in dirnames_dst]

In [ ]:
for i, dirpath in enumerate(dirpaths_dst):
    if not os.path.exists(dirpath): os.makedirs(dirpath)

In [ ]:
for i in range(6):
    relocate_samples(ids[i], filetypes[i], dirpaths_src[i], dirpaths_dst[i])

# Determine class ratios

Which proportion of all pixels in a dataset is taken up by each class?

In [ ]:
dataset_dirpath = r"datasets"
dataset_dirname = "dataset_wbg_v4"
labels_dirpath = os.path.join(dataset_dirpath, dataset_dirname, "labels")

labels_filetype = ".png"
label_filenames = [file for file in os.listdir(labels_dirpath) if file.lower().endswith(labels_filetype)]

In [ ]:
b = time.time()

count = np.zeros(18)
for i, filename in enumerate(label_filenames):
    filepath = os.path.join(labels_dirpath, filename)
    labelimg = cv2.imread(filepath)
    
    print("Read {} files.".format(str(i+1)), end = "\r")
    
    count += np.bincount(labelimg[:,:,0].reshape(-1), minlength = 18)

ratios = np.multiply(count, 1/sum(count))

e = time.time()
print("Read {} files in {} seconds.".format(str(i+1), str(e-b)))

In [ ]:
ratios_dict = dict(zip(range(0,18), ratios))
ratios_dict